# Pandas Cheatsheet

**TOC:**

- [Maximum Columsn to Show](#max_columns)
- [Access Columns Themselves](#access_column)
- [Change Columns Themselves](#change_column)
- [Change Index](#change_index)
- [Change Index Content](#change_index_content)
- [Drop Columns](#drop_column)
- [Drop Rows](#drop_rows)
- [Split Row](#split_row)
- [Fill NA](#fill_na)
- [Logic and/or/not](#logic_and_or_not)
- [isin](#isin)
- [Change Data Type](#change_type)
- [Multiple Replace](#mul_replace)
- [Normalization](#normalization)


Ignore warnings globally:

In [ ]:
import warnings
warnings.simplefilter('ignore')

Load Data

In [ ]:
import numpy as np
import pandas as pd

data = pd.read_csv('data/pokemon.csv')

Pandas can read zipped file directly. E.g., if pokemon.csv is zipped as pokemon.zip, pandas still can load the data as **pd.read_csv('data/pokemon.zip')** directly. Actually, it supports quite a few compression mechanism. Refer to the API reference for details.

While loadding data, datetime can also be parsed directly as **pd.read_csv('data/prices.zip', parse_dates=['date'])** for short(this can be done as data['date'] = date['date'].astype(np.datetime64) after loading)

## Maximum Columns to Show <a id='max_columns'></a>

**When the num. of columsn exceed the allowed num., columns will be folded:**

In [ ]:
pd.set_option('max_columns', 5)
data.head()

**When max_columns is set as None, all columns will be shown:**

In [ ]:
pd.set_option('max_columns', None)
data.head()

## Access Columns Themselves <a id='access_column'></a>

In [ ]:
data.columns

In [ ]:
for col in data.columns:
    print(col)

## Change Columns Themselves <a id='change_column'></a>

In [ ]:
data.columns = data.columns.str.lower().str.replace(' ', '_')
data.head()

## Change Index <a id='change_index'></a>

**By default, index is RangeIndex, which works like a list in python:**

In [ ]:
data.index[0:10]

In [ ]:
data.index.max()

**Use another columne as index:**

In [ ]:
data.set_index('name', inplace=True)

In [ ]:
data.head()

**Revert back to default index style again:**

In [ ]:
data.reset_index(inplace=True)

In [ ]:
data.head()

## Change Index Content <a id='change_index_content'></a>

In [ ]:
data.head()

In [ ]:
data.set_index('name', inplace=True)

In [ ]:
data.head()

**Reassign index to new content to change its original content:**

In [ ]:
data.index = data.index.str.replace(".*(?=Mega)", "")

In [ ]:
data.head()

## Drop Columns <a id='drop_column'></a>

In [ ]:
data.drop(['#'], axis=1, inplace=True)

In [ ]:
data.head()

## Drop Rows <a id='drop_rows'></a>

In [ ]:
data.info()

In [ ]:
data_withoutna = data.dropna()
data_withoutna.info()

## Split One Row into Multiple Rows <a id='split_row'></a>

In [ ]:
samples = pd.DataFrame([["a1,b1,c1", 1, "t1"],["a2", 2, "t2"],["a3,a4", 3, "t3"]])
samples.columns = ["name", "age", "order"]
samples.set_index("order", inplace=True)

In [ ]:
samples

In [ ]:
s = samples.name.str.split(',').apply(pd.Series, 1).stack()
s

In [ ]:
s.index = s.index.droplevel(-1)
s

In [ ]:
s.name = "name"

In [ ]:
s

In [ ]:
del samples["name"]
samples

In [ ]:
samples = samples.join(s)

In [ ]:
samples

## Fill NA <a id='fill_na'></a>

In [ ]:
data.info()

In [ ]:
data[data['type_2'].isna()].head(10)

In [ ]:
data['type_2'].fillna(method='ffill', inplace=True)

In [ ]:
data.loc[['Charmander', 'Charmeleon']]

In [ ]:
data.info()

## Logic and/or/not <a id='logic_and_or_not'></a>

**And**

In [ ]:
data.loc[(data['type_1'] == 'Normal') & (data['type_2'] == 'Rock')].head()

**Or**

In [ ]:
data.loc[(data['type_1'] == 'Normal') | (data['type_1'] == 'Water')].head()

**Not**

In [ ]:
data.loc[~ (data['type_1'] == 'Normal')].head()

## isin <a id='isin'></a>

In [ ]:
data.loc[data['type_1'].isin(['Normal', 'Grass'])].head()

*The above is eqal to the below:*

In [ ]:
data[data['type_1'].isin(['Normal', 'Grass'])].head()

## Change Data Type <a id='change_type'></a>

In [ ]:
data_copy = data.copy()

In [ ]:
data_copy.head()

In [ ]:
type(data_copy['total'][0])

In [ ]:
data_copy['total'] = data_copy['total'].astype(str)

In [ ]:
type(data_copy['total'][0])

## Multiple Replace <a id='mul_replace'></a>

In [ ]:
data_copy = data.copy()

In [ ]:
data_copy.head()

In [ ]:
data_copy['type_1'].replace(['Grass', 'Fire'], ['Tree', 'Storm'], inplace=True)

In [ ]:
data_copy.head()

## Change Values based on Dict Map <a id='dict_map'></a>

In [ ]:
df = data_copy.head(20)

In [ ]:
df.type_1

**Method 1**

In [ ]:
df_map = pd.DataFrame([["Water", "Sea"], ["Tree", "Forest"], ["Storm", "Hail"]])
df_map.columns = ["type1", "new_type1"]
df_map

In [ ]:
df["new_type1"] = df.type_1.map(df_map.set_index("type1").new_type1, na_action='ignore')

In [ ]:
df

**Method 2**

Nonexistent keys can be handled gracefully with the use of lambda

In [ ]:
d_map = {"Water": "Sea", "Tree": "Forest", "Storm": "Hail"}
d_map

In [ ]:
df['new_type1'] = df.type_1.map(lambda x: d_map.get(x) if d_map.get(x, "") != "" else x)

In [ ]:
df

## Normalization <a id='normalization'></a>

In [ ]:
data_copy = data.copy()
data.head()

In [ ]:
attack = data["attack"]
attack_norm1 = (attack - attack.mean())/attack.std()
attack_norm2 = (attack - attack.min())/(attack.max()-attack.min())
data_copy["attack_n1"] = attack_norm1
data_copy["attack_n2"] = attack_norm2

In [ ]:
data_copy.head()